In [ ]:
# standard modules
import io
import json
import os
from pathlib import Path
import time

# specialized modules
import ee
import geemap
import geopandas as gpd
from pathlib import Path
from tqdm import tqdm

# initialize the Earth Engine module.
ee.Initialize(project='trinity-438000')


# read AOI
aoi = Path().cwd() / 'cachuma.geojson'


/home/michael/miniforge3/envs/ee/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
aoi = gpd.read_file(aoi_path).to_crs(4326)
gee_json = json.loads(aoi[['geometry']].to_json())
gee_aoi = geemap.geojson_to_ee(gee_json)

# inspect the GeoJSON as an EEObject through geemap.
test_map = geemap.Map()
test_map.centerObject(gee_aoi, 9) # Centering on the AOI instead of an undefined 'region'
test_map.addLayer(gee_aoi, {}, 'AOI')

test_map


Map(center=[-23.623264544012432, -66.74214501219706], controls=(WidgetControl(options=['position', 'transparen…

In [3]:
# get extent
minx, miny, maxx, maxy = aoi.total_bounds

verts = [[
[minx, miny],
[minx, maxy],
[maxx, maxy],
[maxx, miny],
[minx, miny]
]]

# make normal float from np.float63
verts = [[float(x), float(y)] for x, y in verts[0]]

verts

[[-66.81598222918733, -23.74861602756859],
 [-66.81598222918733, -23.497981330854298],
 [-66.66830779520778, -23.497981330854298],
 [-66.66830779520778, -23.74861602756859],
 [-66.81598222918733, -23.74861602756859]]

In [4]:
# date range
START = ee.Date('2023-01-01')
END = ee.Date('2025-11-01')

# date and geographic filter
col_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.date(START, END),
)

# naip
dw_col = ee.ImageCollection('USDA/NAIP/DOQQ').filter(col_filter)


In [ ]:
def mask_s2_clouds(image):
  '''Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  '''
  qa = image.select('QA60')

  # bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


tasks = []


for start in tqdm(['2018-07-01', '2024-02-01']):

    end = start.advance(23, 'month')

    date_filter = ee.Filter.And(
        ee.Filter.geometry(ee.Geometry.Polygon(verts)),
        ee.Filter.date(start, end)
    )

    s2_median = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filter(date_filter)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35))
        .map(mask_s2_clouds)
        .median()
    )

    export_params = {
        'image': s2_median.clip(gee_aoi),
        'description': f's2_cachuma_median_{year}',
        'folder': 'nr218',
        'fileNamePrefix': f's2_cachuma_median_{year}',
        'scale': 10,
        'region': gee_aoi.geometry(),
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e12
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    tasks.append(task)


100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


In [16]:
for task in tasks:
    info = task.status()
    print(f"{info['description']}: {info['state']}", info.get('progress', ''))


s2_susques_max_2025: RUNNING 
s2_susques_median_2025: COMPLETED 


In [18]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(date_filter)

latest_date = ee.Date(collection.aggregate_max('system:time_start'))

latest_date